<a href="https://colab.research.google.com/github/kotalaraghava/END-2.0---school-of-ai/blob/master/Session6Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [101]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/ai/data/tweets.csv')
df.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [102]:
df.shape

(1364, 2)

In [103]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [104]:
# Import Library
import random
import torch, torchtext
# from torchtext import data 
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [105]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True, init_token='<sos>', eos_token='<eos>')
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [106]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [107]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [108]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [109]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [110]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [111]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Romney',
  'agree',
  ':',
  'Admit',
  'women',
  'to',
  'Augusta',
  'golf',
  'club',
  ':',
  'US',
  'President',
  'Barack',
  'Obama',
  'believes',
  'women',
  'should',
  'be',
  'allowe',
  '...',
  'http://t.co/PVKrepqI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [112]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [113]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4653
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [114]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [115]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [116]:
import os, pickle
import pdb
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

**Encoder**

In [130]:
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
         # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)

        
        
        
        
    def forward(self, src, srcLength):
        
        #src = [src len, batch size]
        # import pdb
        # pdb.set_trace()
        embedded = self.embedding(src)

        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.encoder(embedded)

        #outputs = [batch size, src len,  hid dim * n directions]
        #hidden = [ n layers * n directions, batch size, hid dim]
        hidden_output = hidden.permute(1, 0, 2)
        #[batch_size, num_layers, hid_dim]
        print(f"Encoder output : {outputs.shape}, Encoder hidden shape : {hidden.shape}")        
        return hidden_output

**Decoder**

In [131]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()

        
         # LSTM layer
        self.decoder = nn.LSTM(input_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
        
    def forward(self, src):
        
        outputs, (hidden, cell) = self.decoder(src)

        #outputs = [batch size, src len,  hid dim * n directions]
        #hidden = [batch size, n layers * n directions,  hid dim]

        dense_outputs = self.fc(hidden[0])
        print(f"Decoder output : {outputs.shape}, Decoder hidden shape :{hidden.shape}")
        return dense_outputs

**ENCODER-DECODER_MODEL**

In [132]:
class EncoderDecoderModel(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder

  def forward(self, src, src_length):

    encoder_output = self.encoder(src, src_length)
    # pdb.set_trace()
    decoder_output = self.decoder(encoder_output)
                          
    return decoder_output

**Define hyperparameters**

In [133]:
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = len(Label.vocab)
num_layers =1
dropout = 0.2
dec_input_nodes = num_hidden_nodes


enc = Encoder(size_of_vocab, embedding_dim, num_hidden_nodes, num_layers, dropout)
dec = Decoder(dec_input_nodes, num_hidden_nodes, num_output_nodes, num_layers, dropout)

model = EncoderDecoderModel(enc, dec)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [134]:
print(model)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

EncoderDecoderModel(
  (encoder): Encoder(
    (embedding): Embedding(4653, 300)
    (encoder): LSTM(300, 100, batch_first=True, dropout=0.2)
  )
  (decoder): Decoder(
    (decoder): LSTM(100, 100, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=100, out_features=3, bias=True)
  )
)
The model has 1,637,803 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [135]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [136]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [137]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [138]:
N_EPOCHS = 11
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Encoder output : torch.Size([32, 27, 100]), Encoder hidden shape : torch.Size([1, 32, 100])
Decoder output : torch.Size([32, 1, 100]), Decoder hidden shape :torch.Size([1, 32, 100])
Encoder output : torch.Size([32, 31, 100]), Encoder hidden shape : torch.Size([1, 32, 100])
Decoder output : torch.Size([32, 1, 100]), Decoder hidden shape :torch.Size([1, 32, 100])
Encoder output : torch.Size([32, 18, 100]), Encoder hidden shape : torch.Size([1, 32, 100])
Decoder output : torch.Size([32, 1, 100]), Decoder hidden shape :torch.Size([1, 32, 100])
Encoder output : torch.Size([32, 28, 100]), Encoder hidden shape : torch.Size([1, 32, 100])
Decoder output : torch.Size([32, 1, 100]), Decoder hidden shape :torch.Size([1, 32, 100])
Encoder output : torch.Size([32, 29, 100]), Encoder hidden shape : torch.Size([1, 32, 100])
Decoder output : torch.Size([32, 1, 100]), Decoder hidden shape :torch.Size([1, 32, 100])
Encoder output : torch.Size([32, 16, 100]), Encoder hidden shape : torch.Size([1, 32, 100]

## Model Testing

In [139]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [140]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

Encoder output : torch.Size([1, 15, 100]), Encoder hidden shape : torch.Size([1, 1, 100])
Decoder output : torch.Size([1, 1, 100]), Decoder hidden shape :torch.Size([1, 1, 100])


'Positive'